In [72]:
!pip install -q pyomo

In [73]:
from pyomo.environ import *

In [74]:
import numpy as np

In [75]:
model2 = ConcreteModel()

In [76]:
N = 10 
M = 6 
K = 3 

In [77]:
lb=np.array([0,0,0,0,0,0,0,0,0,0])
ub=np.array([1,1,1,1,1,1,1,1,1,1]) 
obj_coef_c = np.array([4,3,6,5,2,6,4,8,8,6]) 
constr_coef_A_list_less_than = [[0,0,1,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,1],[0,0,1,0,0,0,0,0,1,0],[1,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1],[0,0,0,0,0,0,1,1,0,1]]

constr_coef_A_less_than = np.array(constr_coef_A_list_less_than)

constr_coef_A_list_grt_than = [[0,0,0,0,1,0,0,0,0,1],[1,1,1,1,0,0,0,0,0,0],[4,3,6,5,2,6,4,8,8,6]]

constr_coef_A_grt_than = np.array(constr_coef_A_list_grt_than)

constr_rhs_b_less_than=np.array([1,1,1,2,1,2])

constr_rhs_b_grt_than=np.array([2,1,26,4])


row_indices_less=np.arange(M)
row_indices_grt=np.arange(K)

print(row_indices_less)
print(row_indices_grt)
col_indices = np.arange(N)
print(col_indices)
print(constr_coef_A_list_less_than)
print(constr_coef_A_grt_than)
print(obj_coef_c)

[0 1 2 3 4 5]
[0 1 2]
[0 1 2 3 4 5 6 7 8 9]
[[0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1]]
[[0 0 0 0 1 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [4 3 6 5 2 6 4 8 8 6]]
[4 3 6 5 2 6 4 8 8 6]


In [78]:
model2.x = Var(col_indices, domain = Binary)
model2.constraints = ConstraintList()

In [79]:
for i in col_indices:
  model2.x[i].setlb(lb[i])
  model2.x[i].setub(ub[i])

In [80]:
for i in row_indices_less:
  model2.constraints.add(sum(constr_coef_A_less_than[i][j]*model2.x[j] for j in col_indices) <= constr_rhs_b_less_than[i])

for i in row_indices_grt:
  model2.constraints.add(sum(constr_coef_A_grt_than[i][j]*model2.x[j] for j in col_indices) >= constr_rhs_b_grt_than[i])

In [81]:
model2.objective = Objective(expr =summation(obj_coef_c,model2.x), sense=maximize)
model2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : Fa

In [82]:
!apt-get install -y -qq coinor-cbc

In [83]:
opt_cbc = SolverFactory('cbc')

In [84]:
result = opt_cbc.solve(model2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 33.0
  Upper bound: 33.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.023067951202392578
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.023067951202392578


In [85]:
print('\nObjective = ', model2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model2.x[i].value)

print('\nConstraints')
model2.constraints.display()


Objective =  33.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0

Constraints
constraints : Size=9
    Key : Lower : Body : Upper
      1 :  None :  1.0 :   1.0
      2 :  None :  1.0 :   1.0
      3 :  None :  1.0 :   1.0
      4 :  None :  2.0 :   2.0
      5 :  None :  1.0 :   1.0
      6 :  None :  2.0 :   2.0
      7 :   2.0 :  2.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  26.0 : 33.0 :  None


In [86]:
model2.y= Var(col_indices)

for i in col_indices:
  model2.y[i].setlb(lb[i])
  model2.y[i].setub(ub[i])

In [87]:
for i in row_indices_less:
  model2.constraints.add(sum(constr_coef_A_less_than[i][j]*model2.y[j] for j in col_indices) <= constr_rhs_b_less_than[i])

for i in row_indices_grt:
  model2.constraints.add(sum(constr_coef_A_grt_than[i][j]*model2.y[j] for j in col_indices) >= constr_rhs_b_grt_than[i])


In [88]:
model2.objective.deactivate()

for i in range(1, 10) :
  model2.constraints[i].deactivate()

model2.new_objective = Objective(expr =summation(obj_coef_c,model2.y), sense=maximize)

model2.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   18 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 : 

In [89]:
result = opt_cbc.solve(model2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 33.0
  Upper bound: 33.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 11
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.04539346694946289
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04539346694946289


In [90]:
print('\nObjective = ', model2.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('y[',i,'] = ', model2.y[i].value)

print('\nConstraints')
model2.constraints.display()


Objective =  33.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  0.0
y[ 6 ] =  0.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  1.0

Constraints
constraints : Size=18
    Key : Lower : Body : Upper
     10 :  None :  1.0 :   1.0
     11 :  None :  1.0 :   1.0
     12 :  None :  1.0 :   1.0
     13 :  None :  2.0 :   2.0
     14 :  None :  1.0 :   1.0
     15 :  None :  2.0 :   2.0
     16 :   2.0 :  2.0 :  None
     17 :   1.0 :  2.0 :  None
     18 :  26.0 : 33.0 :  None


In [91]:
for i in range(1, 10) :
  model2.constraints[i].activate()

for i in range(10, 19) :
  model2.constraints[i].deactivate()

model2.x10 = Var(domain = Binary)

model2.constraints.add(expr = model2.x[4]+ model2.x[5] + model2.x10 <= 25)

model2.new_objective.deactivate()
model2.new_objective_2 = Objective(expr =summation(obj_coef_c,model2.x) + 12*model2.x10 , sense=maximize)

In [92]:
model2.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

3 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True : Binary
          1 :     0 :   0.0 :     1 : False :  True : Binary
          2 :     0 :   0.0 :     1 : False :  True : Binary
          3 :     0 :   1.0 :     1 : False :  True : Binary
          4 :     0 :   1.0 :     1 : False :  True : Binary
          5 :     

In [93]:
result = opt_cbc.solve(model2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03366255760192871
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.03366255760192871


In [94]:
print('\nObjective = ', model2.new_objective_2())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model2.x[i].value)

print('x10 =', model2.x10.value)

print('\nConstraints')
model2.constraints.display()


Objective =  45.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x10 = 1.0

Constraints
constraints : Size=19
    Key : Lower : Body : Upper
      1 :  None :  1.0 :   1.0
      2 :  None :  1.0 :   1.0
      3 :  None :  1.0 :   1.0
      4 :  None :  2.0 :   2.0
      5 :  None :  1.0 :   1.0
      6 :  None :  2.0 :   2.0
      7 :   2.0 :  2.0 :  None
      8 :   1.0 :  2.0 :  None
      9 :  26.0 : 33.0 :  None
     19 :  None :  2.0 :  25.0


In [95]:
model2.x11 = Var(domain = Reals)

model2.x11.setlb(0)
model2.x11.setub(1)

In [96]:
model2.new_objective_2.deactivate()

for i in range(1, 10) :
  model2.constraints[i].deactivate()

for i in range(10, 19) :
  model2.constraints[i].activate()  

model2.new_objective_3 = Objective(expr =summation(obj_coef_c,model2.y) + 12*model2.x11 , sense=maximize)

model2.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

4 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     

In [97]:
result = opt_cbc.solve(model2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 3
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03400874137878418
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.03400874137878418


In [98]:
print('\nObjective = ', model2.new_objective_3())

print('\nDecision Variables')
for i in col_indices:
  print('y[',i,'] = ', model2.y[i].value)

print('x11 = ', model2.x11.value)

print('\nConstraints')
model2.constraints.display()


Objective =  45.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  0.0
y[ 6 ] =  0.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  1.0
x11 =  1.0

Constraints
constraints : Size=19
    Key : Lower : Body : Upper
     10 :  None :  1.0 :   1.0
     11 :  None :  1.0 :   1.0
     12 :  None :  1.0 :   1.0
     13 :  None :  2.0 :   2.0
     14 :  None :  1.0 :   1.0
     15 :  None :  2.0 :   2.0
     16 :   2.0 :  2.0 :  None
     17 :   1.0 :  2.0 :  None
     18 :  26.0 : 33.0 :  None
     19 :  None :  0.0 :  25.0


answer 1>
objective function
max 4x0+3x1+6x2+5x3+2x4+6x5+4x6+8x7+8x8+6x9
subject to
x2+x7<=1
x1+x9<=1
x2+x8<=1
x0+x1+x2+x3<=2
x5+x9<=1
x6+x7+x9<=2
x0+x1+x2+x3+x4+x5+x6+x7+x8+x9<=8
x9+x4>=2
x0+x1+x2+x3>=1
4x0+3x1+6x2+5x3+2x4+6x5+4x6+8x7+8x8+6x9>=26
x0+x1+x2+x3+x4+x5+x6+x7+x8+x9>=4

3>

Objective =  33.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0

4>

Objective =  33.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  0.0
y[ 6 ] =  0.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  1.0

5> we can solve MILP problem by rounding off the solution of the corresponding lp problem 
above lp problem is lp relaxation 
if the optimal  solution of the lp relaxation is feasible for the MILP, then it is optimal for original problem

6>change in the value of objective function
33 to 45
and decision variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[10] = 1.0

7>
Objective =  45.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[10] = 1.0

8>

Objective =  45.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  0.0
y[ 6 ] =  0.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  1.0
x[ 10] =  1.0
